In [1]:
import numpy as np

class Portfolio:
    def __init__(self):
        self.symbol_list = ['BTC-USDT', 'ETH-BTC']
        
        self.current_positions = dict((s, 0.0) for s in self.symbol_list)
        self.all_positions = self._construct_all_positions()
        
        self.current_holdings = self._construct_current_holdings()
        self.all_holdings = self._construct_all_holdings()
        
    def _construct_all_positions(self):
        position = dict((s, 0.0) for s in self.symbol_list)
        position['datetime'] = 'start_date'
        return [position]
    
    def _construct_current_holdings(self):
        holding = {
            s: {
                'open_price': 0.0,
                'current_value': 0.0,
                'trailing_sl': 0.0
            }
            for s in self.symbol_list
        }
        holding['cash'] = holding['total'] = 10
        holding['fees'] = 0.0
        
        return holding
    
    def _construct_all_holdings(self):
        holding = self._construct_current_holdings()
        holding['datetime'] = 'start_date'
        
        return [holding]

In [2]:
portfolio = Portfolio()

In [3]:
portfolio.current_holdings

{'BTC-USDT': {'open_price': 0.0, 'current_value': 0.0, 'trailing_sl': 0.0},
 'ETH-BTC': {'open_price': 0.0, 'current_value': 0.0, 'trailing_sl': 0.0},
 'cash': 10,
 'total': 10,
 'fees': 0.0}

In [4]:
portfolio.all_holdings

[{'BTC-USDT': {'open_price': 0.0, 'current_value': 0.0, 'trailing_sl': 0.0},
  'ETH-BTC': {'open_price': 0.0, 'current_value': 0.0, 'trailing_sl': 0.0},
  'cash': 10,
  'total': 10,
  'fees': 0.0,
  'datetime': 'start_date'}]

In [5]:
portfolio.current_positions

{'BTC-USDT': 0.0, 'ETH-BTC': 0.0}

In [6]:
portfolio.all_positions.append({
    **portfolio.current_positions,
    'datetime': 'current_date'
})

In [7]:
portfolio.all_positions

[{'BTC-USDT': 0.0, 'ETH-BTC': 0.0, 'datetime': 'start_date'},
 {'BTC-USDT': 0.0, 'ETH-BTC': 0.0, 'datetime': 'current_date'}]

In [8]:
def trailing_sl(open_price, current_value, pct_sl, pct_tp):
    returns = (current_value / open_price) - 1
    trailing_sl = None
    if returns >= pct_tp:
        trailing_sl = current_value*(1-pct_sl)
    else:
        trailing_sl = open_price*(1-pct_sl)
        
    return trailing_sl

In [211]:
trailing_sl(100, 110, 0.02, 0.05)

<function __main__.trailing_sl.<locals>.compute_sl(current_value)>

In [9]:
def trailing_sl(symbol, open_price, pct_sl, pct_tp):
    trailing_dict = dict()
    trailing_dict[symbol] = open_price*(1-pct_sl)
    
    def compute_sl(current_value):
        returns = (current_value / open_price) - 1
        current_trailing_sl = 0.0
        if returns >= pct_tp:
            current_trailing_sl = current_value*(1-pct_sl)
        
        trailing_dict[symbol] = max(trailing_dict[symbol], current_trailing_sl)
        return trailing_dict[symbol]
    
    return compute_sl

In [10]:
init_trailing = dict((symbol, 0.0) for symbol in ['BTC-USDT', 'ETH-BTC'])

In [11]:
init_trailing['ETH-BTC'] = trailing_sl('ETH-BTC', 100, 0.02, 0.05)

In [12]:
init_trailing['BTC-USDT'] = trailing_sl('BTC-USD', 130, 0.02, 0.05)

In [13]:
init_trailing

{'BTC-USDT': <function __main__.trailing_sl.<locals>.compute_sl(current_value)>,
 'ETH-BTC': <function __main__.trailing_sl.<locals>.compute_sl(current_value)>}

470.4

In [217]:
init_trailing['BTC-USDT'](900)

882.0

In [221]:
init_trailing['ETH-BTC'](480)

470.4